# intracranial EEG preprocessing pipeline using MNE-python package

In [1]:
# Import relevant packages
import os
import numpy as np
import mne
import pandas as pd 



import matplotlib.pyplot as plt
import matplotlib
plt.ion()

In [2]:
# Define data file locations etc
homeDir = '/Users/yvonnefonken/Documents/Oxford/Research Projects/intracranial'
os.chdir(homeDir)

# set montage file
#montage = homeDir + "montage/m.elp"
subjects = ['UCL02'] # Only analyzing 18-30 for now

filenames = ['YF_2802b.edf']

metadataL = ['1L_iEEG.csv']

metadataO = ['1O_iEEG.csv']

#version = []

#['S00_experimenter', 'S01', 'S02', 'S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S09', 'S10', 'S11', 'S12', 'S13', 'S14',
           #'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22', 'S23']


In [ ]:
dfO

In [ ]:
print(len(dfL))
print(len(dfO))

# number of onsets in Learning
print(len(dfL) * 3)
print(len(dfO) * 2)
print(len(dfL)* 3  + len(dfO) *2)

In [ ]:
# Set parameters

#filtering settings ERP data
lp = 0.1 # 
hp = 200

# downsample to this frequency
sfreq = 250

# filtering settings time-frequency data
#lpTF = 1
#hpTF = 100

#sfreqTF = 100




In [ ]:
mne.io.read_raw_edf?

In [3]:
# Load data
sub = 1 # 28 is in two files, need to fix -- Learning file for sub 30 fixed. 27,21, 19 has weird looking PSD
subInd = sub -1

raw_datafile = os.path.join(homeDir, subjects[subInd], filenames[subInd])
raw = mne.io.read_raw_edf(raw_datafile, preload=True, exclude = ['MKR1+', 'MKR2+'])#, exclude = ['xyz+', 'PULS+', 'BEAT+', 'SpO2', 'MKR2+', 'ches+', 'abd+', 'EEG EKG 1', 'EEG EKG 2', 'EEG EKG 3', 'EEG EKG4', 'MKR1+'])
#mne.io.set_eeg_reference(raw, ['RM'])



Extracting EDF parameters from /Users/yvonnefonken/Documents/Oxford/Research Projects/intracranial/UCL02/YF_2802b.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5064063  =      0.000 ...  4945.374 secs...


In [4]:
dfL = pd.read_csv(homeDir + '/' + 'UCL02' + '/' + '2L_iEEG.csv')
dfO = pd.read_csv(homeDir + '/' + 'UCL02' + '/' + '2O_iEEG.csv')

In [5]:
# get channel names to exclude to only look at analog MKR channels
excludeChans = raw.info['ch_names']

rawAn = mne.io.read_raw_edf(raw_datafile, preload = True, exclude = excludeChans)

#save memory while extracting analog channels


Extracting EDF parameters from /Users/yvonnefonken/Documents/Oxford/Research Projects/intracranial/UCL02/YF_2802b.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5064063  =      0.000 ...  4945.374 secs...


In [ ]:
# look at analog channels to get more precise timings of differen blocks, to later split up data to decrease RAM usage
%matplotlib notebook
# lala
fig = rawAn.plot(duration = 90)#, order = np.arange(0,125))
fig.set_figheight(10)
fig.set_figwidth(10)

In [6]:
# extract onsets and offsets of analog channel triggers

ana = rawAn._data[1,:] - rawAn._data[0,:]

print(np.mean(ana))
print(np.max(ana))
print(np.std(ana))
sd = np.std(ana)
ana = (ana - np.mean(ana))
anaFilt = ana.copy()
#rawAn._data = np.concatenate((rawAn._data, ana.T))
#rawAn.info['ch_names'].append('convertAna')
#rawAn._data[0,:] = ana

onset = 0
eventOnset = []
eventOffset = []

#ana = np.convolve(ana, np.ones((20,))/20, mode='valid') # clean up signal -- note: may need to check for significant time-shifts
threshold = 0.001
#rawAn._data = np.concatenate((rawAn._data, ana))
#rawAn.info['ch_names'].append('filtAna')
#rawAn._data[1,:] = ana
winSize = 50

for i in range(winSize,len(ana) - winSize):
    
    # if analog signal is above the threshold, and no onset has been recorded
    if np.abs(ana[i]) > threshold and onset == 0:
        # Record onset if this is a sustained threshold crossing (avoing blips to be recorded as events)
        if np.sum(np.abs(ana[(i-winSize):i]) < threshold) > 40 and np.sum(np.abs(ana[i:(i+winSize)] )> threshold) >15: # record onset only if previous 50 samples were below threshold
            onset = 1
            eventOnset.append(i)
            
    # if an onset has been recorded detect when the analog channel goes back to baseline
    elif np.abs(ana[i]) < threshold and onset == 1:
        if np.sum(np.abs(ana[i:(i+winSize)] )<threshold) > 40 and np.sum(np.abs(ana[(i-winSize):i]) > threshold) >10:
            onset = 0
            eventOffset.append(i)

print('counted ' + str(len(eventOnset)) + ' event onsets')
print('counted ' + str(len(eventOffset)) + ' event offsets')

5.4853114095403154e-05
0.0032499519340810253
0.00041935752006440704
counted 598 event onsets
counted 598 event offsets


In [ ]:
# plot analog channel with event onset and offsets

# create events array 
%matplotlib notebook

events = np.zeros((len(eventOnset) + len(eventOffset), 3))
events[:,0] = np.concatenate((np.asarray(eventOnset), np.asarray(eventOffset)))
events[0:len(eventOnset), 2] = 1.0
                             

fig = rawAn.plot(duration = 60, events = events)
fig.set_figheight(7)
fig.set_figwidth(10)
#for i in eventOnset:
    
    #plt.axvline(x=i, color = 'gray')



In [7]:
%matplotlib qt
eventDur = np.asarray(eventOffset) - np.asarray(eventOnset)
plt.hist(eventDur, bins = 100) #, bins = [0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500])

(array([  2.,   5.,   0.,   1.,   1.,  22., 286.,  19.,   2.,   0.,   0.,
          2.,   0.,   0.,   0.,   0.,   4.,   1.,   0.,   0.,   2.,   0.,
          0.,   0., 122.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         65.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,
          1.,   2.,   0.,   0.,   0.,   0.,   4.,  52.,   0.,   1.,   0.,
          1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,
          1.]),
 array([  14.  ,   23.88,   33.76,   43.64,   53.52,   63.4 ,   73.28,
          83.16,   93.04,  102.92,  112.8 ,  122.68,  132.56,  142.44,
         152.32,  162.2 ,  172.08,  181.96,  191.84,  201.72,  211.6 ,
         221.48,  231.36,  241.24,  251.12,  261.  ,  270.88,  280.76,
         290.64,  300.52,  310.4 ,

In [ ]:
# plot analog channel with event onset and offsets

# create events array 
%matplotlib notebook

events = np.zeros((len(eventsLBeep) + len(eventsOBeep), 3))
events[:,0] = np.concatenate((np.asarray(eventsLBeep), np.asarray(eventsOBeep)))
#events[0:len(eventsOBeep), 2] = 1.0
                             

fig = rawAn.plot(duration = 45, events = events)
fig.set_figheight(3)
fig.set_figwidth(10)

In [ ]:
A = np.zeros((len(eventOnset), 2))
A[:,0] = eventOnset
A[:,1] = eventDur
for i in range(len(eventOnset)):
    print(A[i,0]/sfreq)
    print(A[i,1])

In [8]:
# exclude events outside blocks (i.e. practice and test sounds)

sfreq = rawAn.info['sfreq']
Ltimes = [370, 2800] 
Otimes = [3100, 3920] 

# convert to timestamps
LInd = [0,0]
OInd = [0,0]

LInd[0] = np.round(sfreq*Ltimes[0])
LInd[1] = np.round(sfreq*Ltimes[1])
OInd[0] = np.round(sfreq*Otimes[0])
OInd[1] = np.round(sfreq*Otimes[1])

eventsL = np.zeros((len(eventOnset),3))
eventsO = np.zeros((len(eventOnset),3))
eventsLBeep = []
eventsOBeep = []

Li = 0
Oi = 0
prevStamp = 0

for tstamp, tdur in zip(np.asarray(eventOnset), np.asarray(eventDur)):
    if tstamp < LInd[1] and tstamp > LInd[0]:
        interval = (tstamp-prevStamp)/sfreq
        eventsL[Li, :] = np.asarray([tstamp, tdur, interval])
        if tdur > 50 and tdur < 89:
            if interval > 3.: # only include if the previous beep was more than 2s ago
                eventsLBeep.append(tstamp) # first trial skipped
            #print(tstamp/sfreq)
            #print(tdur)
        Li += 1
        prevStamp = tstamp
    elif tstamp <OInd[1] and tstamp > OInd[0]:
        interval = (tstamp-prevStamp)/sfreq
        eventsO[Oi,:] = np.asarray([tstamp, tdur, interval])
        if tdur > 29 and tdur <110:
            if interval > 1.7:
                eventsOBeep.append(tstamp)
                #print(tstamp/sfreq)
                #print(tdur)
            
            
        prevStamp = tstamp
        Oi += 1
        
print(eventsL.shape)
print(eventsO.shape)
print(Li)
print(Oi)

# add missing trials
learnMissed = np.asarray([1., 2., 3., 1014.0,]) # remove these trials later
# convert to tstamp
learnMissed = learnMissed*sfreq
# add to eventsLBeep
for i in learnMissed: eventsLBeep.append(round(i))
# sort eventsLBeep
eventsLBeep.sort()


OMissed = np.asarray([3217.56, 3714.72, 3864.02, 3871.16, 3876.62])
OMissed = OMissed*sfreq
for i in OMissed: eventsOBeep.append(round(i))
eventsOBeep.sort()

(598, 3)
(598, 3)
308
244


In [28]:
eventOnset

[4029,
 4906,
 15858,
 16711,
 26929,
 29198,
 38758,
 39620,
 40874,
 165425,
 166215,
 346132,
 356317,
 357194,
 381162,
 382025,
 383278,
 393523,
 395629,
 406284,
 407324,
 408376,
 423745,
 424608,
 426002,
 436035,
 437075,
 438151,
 451120,
 451983,
 453377,
 463614,
 464477,
 477635,
 479716,
 491266,
 492294,
 493440,
 504201,
 505241,
 522655,
 523533,
 534740,
 535617,
 536822,
 548904,
 550985,
 561040,
 562070,
 563121,
 573380,
 575462,
 586756,
 587618,
 588872,
 601636,
 602938,
 616558,
 618827,
 633886,
 634738,
 636133,
 645981,
 647021,
 661292,
 662169,
 662268,
 794303,
 795194,
 796577,
 805378,
 806256,
 807460,
 818723,
 819600,
 820804,
 829271,
 830302,
 841387,
 842405,
 843493,
 853287,
 854140,
 863436,
 864299,
 865553,
 874732,
 876989,
 954104,
 955137,
 956225,
 964514,
 965367,
 966620,
 974837,
 975715,
 976919,
 988878,
 990970,
 998444,
 999321,
 1009043,
 1011125,
 1019827,
 1020705,
 1030458,
 1031321,
 1053142,
 1053995,
 1055389,
 1064971,
 1

In [41]:
RT = []
for i in eventsOBeep:
    for k in range(len(eventOnset)-1):
        if i == eventOnset[k]:
            RT.append((eventOnset[k+1] - eventOnset[k])/sfreq  -0.8
                     )
RT

[0.8416015625,
 1.15703125,
 1.0564453125,
 2.3865234375,
 1.246875,
 1.096484375,
 1.2166015625,
 0.897265625,
 2.2771484375,
 2.1970703125,
 0.9978515625,
 1.19609375,
 1.256640625,
 2.256640625,
 2.4265625,
 2.4236328125,
 2.1375,
 2.256640625,
 2.54765625,
 2.307421875,
 2.15703125,
 2.6775390625,
 2.6765625,
 1.916796875,
 1.19609375,
 2.5671875,
 2.576953125,
 1.5173828125,
 1.4763671875,
 1.5369140625,
 1.1267578125,
 1.1765625,
 2.4568359375,
 2.7771484375,
 2.237109375,
 1.2068359375,
 1.065234375,
 1.397265625,
 2.346484375,
 1.10625,
 1.4568359375,
 0.8865234375,
 2.256640625,
 2.397265625,
 2.5974609375,
 4.170703125,
 2.44609375,
 1.2263671875,
 2.35625,
 2.576953125,
 1.5271484375,
 1.15703125,
 2.2771484375,
 1.7263671875,
 1.1765625,
 1.3875,
 2.897265625,
 1.096484375,
 2.4568359375,
 1.326953125,
 2.7771484375,
 2.22734375,
 2.256640625,
 2.1267578125,
 2.1912109375,
 2.3767578125,
 1.08671875,
 1.246875,
 2.3572265625,
 2.397265625,
 2.647265625,
 1.3376953125,
 1.27

In [23]:
# plot analog channel with event onset and offsets

# create events array 
%matplotlib notebook

events = np.zeros((len(eventsLBeep) + len(eventsOBeep), 3))
events[:,0] = np.concatenate((np.asarray(eventsLBeep), np.asarray(eventsOBeep)))
#events[0:len(eventsOBeep), 2] = 1.0
                             

fig = rawAn.plot(duration = 45, events = events)
fig.set_figheight(3)
fig.set_figwidth(10)

In [59]:
# check timings with logfiles

trialDiff = []


for i in range(1, len(eventsLBeep)):
    trialDiff.append( (eventsLBeep[i] - eventsLBeep[i-1])/sfreq)

trialDiffLog = []
trialDiff.append(0.0)
for i in range(1, len(dfL) ):
    trialDiffLog.append((dfL['trialStamp'][i] - dfL['trialStamp'][i-1])/1000)
trialDiffLog.append(0.3)
timings = pd.DataFrame(data = {'log': trialDiffLog, 'analog': trialDiff})

# add this info to dfL, now it's the time diff between that trial and the next trial. If not the same within a margin, remove that trial
# double check this with timestamps
dfL['logDiff'] = trialDiffLog
dfL['analogDiff'] = trialDiff

dfL['analogTime'] = eventsLBeep
#dfL['analogTime'][1:] = neweventsOBeep[:-1]
dfL['analogTime'] = dfL['analogTime']/sfreq

timings

,log,analog
0,11.3491,1.000000
1,11.9990,1.000000
2,12.2656,369.228516
3,12.0657,12.071289
4,12.4656,12.461914
5,17.0486,17.051758
6,11.9989,12.001953
7,14.7321,14.731445
8,12.1990,12.201172
9,13.6822,13.692383


In [64]:
# check timings with logfiles
#del trialDiff, trialDiffLog
trialDiff = []
badTrigger = [3188. *sfreq, 3190.*sfreq]

neweventsOBeep = []
neweventsOBeep.append(eventsOBeep[0])
for i in range(1, len(eventsOBeep)):
    if (eventsOBeep[i] < badTrigger[1]) & (eventsOBeep[i] > badTrigger[0]): 
        print('bad trial')
    else: 
        trialDiff.append( (eventsOBeep[i] - eventsOBeep[i-1])/sfreq)
        neweventsOBeep.append(eventsOBeep[i])

trialDiffLog = []
trialDiff.append(0.0)
#neweventsOBeep.append(0.0)
#trialDiff.append(0.0)
for i in range(1, len(dfO) ):
    trialDiffLog.append((dfO['trialStamp'][i] - dfO['trialStamp'][i-1])/1000)
trialDiffLog.append(0.3)
timings = pd.DataFrame(data = {'log': trialDiffLog, 'analog': trialDiff})

dfO['logDiff'] = trialDiffLog
dfO['analogDiff'] = trialDiff

RT = []
for i in neweventsOBeep:
    for k in range(len(eventOnset)-1):
        if i == eventOnset[k]:
            RT.append((eventOnset[k+1] - eventOnset[k])/sfreq)

dfO['analogTime'] = neweventsOBeep

dfO['analogTime'] = dfO['analogTime']/sfreq
timings

bad trial


,log,analog
0,6.3161,4.935547
1,5.1996,5.201172
2,5.0995,5.100586
3,6.4828,6.480469
4,5.2663,5.260742
5,5.1495,5.150391
6,5.2662,5.270508
7,4.9496,4.951172
8,6.3495,6.350586
9,6.1328,6.129883


In [65]:
# Combine metadata and events for learning and orienting blocks
eventsL = np.zeros(( len(eventsLBeep), 3))
eventsL[:,0] = eventsLBeep
eventsL[:,2] = np.ones((len(eventsLBeep)))

eventsO = np.zeros(( len(neweventsOBeep), 3))
eventsO[:,0] = neweventsOBeep
eventsO[:,2] =2.0*np.ones((len(neweventsOBeep)))

events = np.concatenate((eventsL, eventsO), axis = 0)
events.astype(int)
df = dfL.append(dfO, ignore_index = True, sort = False)

In [61]:
dfL.keys()

Index(['Unnamed: 0', 'trlNum', 'Condition', 'trialStamp', 'Target1X',
       'Target1Y', 'Target2X', 'Target2Y', 'Target1ID', 'Target2ID',
       'Target1Answer', 'Target2Answer', 'place1X', 'place1Y', 'place2X',
       'place2Y', 'RTselec1', 'RTselec2', 'RTplace1', 'RTplace2', 'selec1X',
       'selec1Y', 'selec2X', 'selec2Y', 'jitterDur', 'Scene', 'subCode',
       'correct', 'correct1', 'correct2', 'nRep', 'pixelError1', 'pixelError2',
       'selecError1', 'selecError2', 'locationForget1', 'locationForget2',
       'correctType', 'BlockType', 'logDiff', 'analogDiff', 'analogTime'],
      dtype='object')

In [63]:
dfL[['Condition', 'correct', 'RTselec1',  'Target1ID', 'Target2ID',  'analogTime' ]]



,Condition,correct,RTselec1,Target1ID,Target2ID,analogTime
0,predicted,1.0,1168.8,2,3,1.000000
1,random,1.0,1379.8,2,1,2.000000
2,predicted,1.0,2316.5,3,1,3.000000
3,predicted,1.0,1891.2,4,2,372.228516
4,random,1.0,1505.5,3,2,384.299805
5,random,1.0,1727.9,1,4,396.761719
6,random,1.0,1231.3,4,1,413.813477
7,predicted,1.0,1704.3,1,2,425.815430
8,predicted,1.0,1348.3,4,1,440.546875
9,random,0.0,2165.3,4,3,452.748047


## Filter data

### Required: filter and downsample data

In [66]:

raw.filter(0.3, 250,  fir_design="firwin")
#raw, events = raw.resample(100, npad='auto', events=events)

#raw.drop_channels(["MKR1+", "MKR2+"])

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 2.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 250.00 Hz
- Upper transition bandwidth: 62.50 Hz (-6 dB cutoff frequency: 281.25 Hz)
- Filter length: 11265 samples (11.001 sec)



<RawEDF  |  YF_2802b.edf, n_channels x n_times : 107 x 5064064 (4945.4 sec), ~4.04 GB, data loaded>

#### Optional: look at PSD after filtering|

In [ ]:
%matplotlib inline
raw.plot_psd(fmax = 40)

## Epoch data

In [67]:


tmin = -3.0
tmax = 12.0 # make it long enough sufficient for TFR analyses




keys     = ['Learning', 'Orienting']

values   = [ 1, 2]

event_ids_target = dict(zip(keys, values))


# Epoch data

epochs = mne.Epochs(raw, events.astype(int), event_ids_target, tmin=tmin, tmax=tmax, preload=True, reject = None, metadata = df) # set reject to None because we will do autoreject

epochs.events[:,1] = range(1,len(epochs.events)+1) #Fill that weird zeros column with the trial numbers

epochs.apply_baseline((None, 0))

del raw

256 matching events found
Applying baseline correction (mode: mean)
Adding metadata with 50 columns
0 projection items activated
Loading data for 256 events and 15361 original time points ...
2 bad epochs dropped
Applying baseline correction (mode: mean)


In [68]:
# Remove trials with clear spikes in HPC, select so trial turns red, hit 'power' button at the end and trials will be removed from epochs
chans = ['EEG laH', 'EEG lpH', 'EEG raH', 'EEG rpH'] # select which probes to look at
elecs = []
for i in range(len(epochs.info['ch_names'])): 
    for k in chans: 
        if k in epochs.info['ch_names'][i]:
            if epochs.info['ch_names'][i] != 'EEG raH10':
                elecs.append(i)

fig = epochs.plot(n_epochs = 3, picks = elecs, n_channels = 30)
fig.set_figheight(8)
fig.set_figwidth(10)

Dropped 52 epochs: 0, 2, 3, 7, 9, 10, 12, 16, 17, 18, 20, 21, 35, 36, 41, 56, 67, 70, 85, 86, 87, 96, 97, 109, 120, 125, 126, 136, 137, 140, 142, 152, 157, 165, 171, 175, 178, 181, 182, 185, 186, 194, 195, 209, 212, 215, 218, 237, 238, 250, 251, 252
Channels marked as bad: []


In [23]:
epochs

<Epochs  |   218 events (all good), -3 - 12 sec, baseline [None, 0], ~2.67 GB, data loaded, with metadata,
 'Learning': 110
 'Orienting': 108>

In [ ]:
# Remove bad channels -- do manually based on above plots -- write code to reject bad channels based on variance, use sub 30 as example
%matplotlib qt
%matplotlib qt
raw.plot()

In [ ]:
# Remove outlier trials -- can only run once, otherwise you start to remove more trials!

# Drop eye movement trials
#epochs.drop(eyeArtifactInd)

axis, keep_idx = plot_AR(epochs, method='gesd')



In [ ]:
epochs = epochs[keep_idx[0]]

In [69]:
# Save ERP data
fileName = 'dataClean-epo.fif'
saveFile = os.path.join(homeDir, subjects[subInd], fileName)
epochs.save(saveFile, overwrite = True, verbose = True)



Overwriting existing file.


In [ ]:
saveFile

In [ ]:
del epochsEye
del raw
del raw2

In [ ]:
epochs.info['ch_names'] 

## Do TFR  -- scratch this for now. Saving data suitable for TFR and ERP analyses instead. 

In [ ]:
# Re-load and re-filter data
subInd = sub -1

raw_datafile = os.path.join(homeDir, subjects[subInd], filenames[subInd])
rawTFR = mne.io.read_raw_eeglab(raw_datafile, montage, eog=['VEOG', 'HEOG'], preload=True)
mne.io.set_eeg_reference(rawTFR, ['RM'])

# Reload events (which are downsampled earlier)

# Housekeeping, filtering, and downsampling
    
# Find the first even with a real trigger code (this is because of the impedence trigger)
#f = np.min(np.where(events[:,-1] < 256))

# crop 10 seconds before the first event
tmin = rawTFR.times[eventsTFR[0][0,0]]-10
if tmin < 0: # if tmin is negative just set it at the beginning
    tmin = rawTFR.times[0] 
# crop 10 seconds after the last event
tmax = rawTFR.times[eventsTFR[0][-1,0]]+10
if tmax > rawTFR.times[-1]: # if tmax is bigger then the max time, just take that
    tmax = rawTFR.times[-1]

# crop it
rawTFR.crop(tmin=tmin, tmax=tmax)

rawTFR.filter(lpTF, hpTF,  fir_design="firwin") 
#rawTFR, events = rawTFR.resample(sfreq, npad='auto', events=events[0]) # Resample later

rawTFR.drop_channels(["RM"])





In [ ]:
# Epoch and Remove ica and bad trials

# Remove eye-artifact ICA's from raw data, re-epoch
#del epochsEye # Clear memory

tmin = -2.0
tmax = 3.0




keys     = ['Blimp']

values   = [ 50]

event_ids_target = dict(zip(keys, values))

# Remove bad ica

rawTFR2 = ica.apply(rawTFR, exclude=ica.exclude).pick_types(eeg=True)

# Epoch data

epochsTFR = mne.Epochs(rawTFR2, eventsTFR[0], event_ids_target, tmin=tmin, tmax=tmax, preload=True, metadata = df) # set reject to None because we will do autoreject

epochsTFR.events[:,1] = range(1,len(epochsTFR.events)+1) #Fill that weird zeros column with the trial numbers

epochsTFR.apply_baseline((None, 0))

# drop eye movement excluded trials
epochsTFR.drop(eyeArtifactInd)
# drop high variance trials
epochsTFR = epochsTFR[keep_idx[0]] # check whether this also updates df -- checked, it does


In [ ]:
# Implement TFR magic
frequencies = np.logspace(*np.log10([3, 45]), num = 15)  # define frequencies of interest
n_cycles = frequencies / 5.  # different number of cycle per frequency
Fs = rawTFR.info['sfreq']  # sampling in Hz
decim = 10
data = epochsTFR.get_data()
print(data.shape)

from mne.time_frequency import tfr_morlet
power = tfr_morlet(epochsTFR, freqs=frequencies,n_cycles=n_cycles, use_fft=False, decim=decim, zero_mean=True, average = True, return_itc = False)
print(power._data.shape)
print ('done!')


In [ ]:
power._data.shape

In [ ]:
# Check data

power.plot_topomap(ch_type='EEG', tmin=0.6, tmax=1.0, fmin=8, fmax=12,
                   baseline=(-0.5, 0), mode='logratio', axes=axis[1],
                   title='Alpha')

power.plot_topomap(ch_type='EEG', tmin=0.8, tmax=1.5, fmin=13, fmax=30,
                   baseline=(-0.5, 0), mode='logratio', axes=axis[1],
                   title='Beta')

In [ ]:
# Save TFR 
fileName = 'testTFR-epo.fif'
saveFile = os.path.join(homeDir, subjects[subInd], fileName)
power.save(saveFile, overwrite = False, verbose = True)

In [ ]:
def gesd(x, alpha = 0.05, p_out = .1, outlier_side = 0):
    import numpy as np
    import scipy.stats
    import copy
    
    '''
    Detect outliers using Generalizes ESD test
    based on the code from Romesh Abeysuriya implementation for OSL
      
    Inputs:
    - x : Data set containing outliers - should be a np.array 
    - alpha : Significance level to detect at (default = .05)
    - p_out : percent of max number of outliers to detect (default = 10% of data set)
    - outlier_side : Specify sidedness of the test
        - outlier_side = -1 -> outliers are all smaller
        - outlier_side = 0 -> outliers could be small/negative or large/positive (default)
        - outlier_side = 1 -> outliers are all larger
        
    Outputs
    - idx : Logicial array with True wherever a sample is an outlier
    - x2 : input array with outliers removed
    
    For details about the method, see
    B. Rosner (1983). Percentage Points for a Generalized ESD Many-outlier Procedure, Technometrics 25(2), pp. 165-172.
    http://www.jstor.org/stable/1268549?seq=1
    '''

    if outlier_side == 0:
        alpha = alpha/2
    
    
    if type(x) != np.ndarray:
        x = np.asarray(x)

    n_out = int(np.ceil(len(x)*p_out))

    if any(~np.isfinite(x)):
        #Need to find outliers only in non-finite x
        y = np.where(np.isfinite(x))[0] # these are the indexes of x that are finite
        idx1, x2 = gesd(x[np.isfinite(x)], alpha, n_out, outlier_side)
        # idx1 has the indexes of y which were marked as outliers
        # the value of y contains the corresponding indexes of x that are outliers
        idx = [False] * len(x)
        idx[y[idx1]] = True

    n      = len(x)
    temp   = x
    R      = np.zeros((1, n_out))[0]
    rm_idx = copy.deepcopy(R)
    lam    = copy.deepcopy(R)

    for j in range(0,int(n_out)):
        i = j+1
        if outlier_side == -1:
            rm_idx[j] = np.nanargmin(temp)
            sample    = np.nanmin(temp)
            R[j]      = np.nanmean(temp) - sample
        elif outlier_side == 0:
            rm_idx[j] = int(np.nanargmax(abs(temp-np.nanmean(temp))))
            R[j]      = np.nanmax(abs(temp-np.nanmean(temp)))
        elif outlier_side == 1: 
            rm_idx[j] = np.nanargmax(temp)
            sample    = np.nanmax(temp)
            R[j]      = sample - np.nanmean(temp)
        
        R[j] = R[j] / np.nanstd(temp)
        temp[int(rm_idx[j])] = np.nan
        
        p = 1-alpha/(n-i+1)
        t = scipy.stats.t.ppf(p,n-i-1)
        lam[j] = ((n-i) * t) / (np.sqrt((n-i-1+t**2)*(n-i+1)))
    
    #And return a logical array of outliers
    idx = np.zeros((1,n))[0]
    idx[np.asarray(rm_idx[range(0,np.max(np.where(R>lam))+1)],int)] = np.nan
    idx = ~np.isfinite(idx)
    
    x2 = x[~idx]

        
    return idx, x2



In [ ]:
def plot_AR(epochs, method = 'gesd', zthreshold = 1.5, p_out = .1, alpha = .05, outlier_side = 1):
    import seaborn as sns
    import pandas as pd
    import scipy.stats

    #Get data, variance, number of trials, and number of channels
    dat     = epochs.get_data()
    var     = np.var(dat, 2)
    ntrials = np.shape(dat)[0]
    nchan   = len(epochs.ch_names)

    #set up the axis for the plots
    x_epos  = range(1,ntrials+1)
    y_epos  = np.mean(var,1)
    y_chans = range(1,nchan+1)
    x_chans = np.mean(var,0)

    #scale the variances
    y_epos  = [x * 10**6 for x in y_epos]
    x_chans = [x * 10**6 for x in x_chans]

    #Get the zScore
    zVar = scipy.stats.zscore(y_epos)

    #save everything in the dataFrame
    df_epos           = pd.DataFrame({'var': y_epos, 'epochs': x_epos, 'zVar': zVar})
    df_chans          = pd.DataFrame({'var': x_chans, 'chans': y_chans})
    
    # Apply the artefact rejection method
    if method == 'gesd':
        idx,x2            = gesd(y_epos, p_out=p_out, alpha=alpha, outlier_side=outlier_side) #use the gesd to find outliers (idx is the index of the outlier trials)
        keepTrials        = np.ones((1,ntrials))[0]
        keepTrials[idx]   = 0
        title = 'Generalized ESD test (alpha=' + str(alpha) + ', p_out=' + str(p_out) + ', outlier_side=' + str(outlier_side) + ')'
    elif method == 'zScore':
        keepTrials        = np.where(df_epos['zVar'] > zthreshold, 0, 1)
        title = 'ZVarience threshold of ' + str(zthreshold)
    elif method == 'none':
        title = 'no additional artefact rejection '
        keepTrials        = np.ones((1,ntrials))[0]
    
    df_epos['keepTrial'] = keepTrials
    df_keeps = df_epos[df_epos['keepTrial'] == 1]
    print(str(ntrials - len(df_keeps)) + ' trials discarded')
    
    # get the clean data
    keep_idx    = np.asarray(np.where(keepTrials),int)
    clean_dat    = np.squeeze(dat[keep_idx])
    
    #recalculate the var for chan
    clean_var    = np.var(clean_dat, 2)
    x_chans_c    = np.mean(clean_var,0)
    x_chans_c    = [x * 10**6 for x in x_chans_c]

    df_chans_c   = pd.DataFrame({'var': x_chans_c, 'chans': y_chans})
    
    
    # Plot everything
    fig, axis = plt.subplots(2, 2, figsize=(12, 12))
    axis[0,0].set_ylim([0, max(y_epos) + min(y_epos)*2])
    axis[0,1].set_xlim([0, max(x_chans)+ min(x_chans)*2])
    axis[1,0].set_ylim([0, max(df_keeps['var'])+ min(df_keeps['var'])*2])
    axis[1,1].set_xlim([0, max(x_chans_c)+ min(x_chans_c)*2])

    axis[0,0].set_title(title)
    sns.scatterplot(x = 'epochs', y = 'var', hue = 'keepTrial', hue_order = [1,0], ax = axis[0,0], data = df_epos)
    sns.scatterplot(x = 'var', y = 'chans', ax = axis[0,1], data = df_chans)
    sns.scatterplot(x = 'epochs', y = 'var', ax = axis[1,0], data =df_keeps)
    sns.scatterplot(x = 'var', y = 'chans', ax = axis[1,1], data = df_chans_c)
    
    
    
    return axis, keep_idx

